# 1. Model Problem
The problem takes the form: find  $u_i : \Omega_i \to \mathbb{R}$  and $u_\Gamma : \Gamma \to \mathbb{R}$ such that
$$
\begin{equation}
	\begin{aligned}
	-\nabla \cdot A_i \nabla u_i &= f_i \quad &&\text{in } \Omega_i \\
	-\nabla_\Gamma \cdot A_\Gamma \nabla_\Gamma u_\Gamma &= f_\Gamma - [\![ n \cdot A \nabla u ]\!] \quad &&\text{on } \Gamma  \\
	n \cdot A \nabla u + B(u - u_\Gamma) &= 0 \quad &&\text{on } \Gamma  \\
	u &= 0 \quad &&\text{on } \partial \Omega 
	\end{aligned}
\end{equation}
$$
Here the jump (or sum) of the normal fluxes is defined by
$$
[\![ n \cdot A \nabla u ]\!] = \sum_{i=1}^{2} n_i \cdot A_i \nabla v_i
$$
We also used the notation
$$n \cdot A \nabla v = \begin{bmatrix} n_1 \cdot A_1 \nabla v_1 \\ n_2 \cdot A_2 \nabla v_2 \end{bmatrix}, \quad v - v_\Gamma = \begin{bmatrix} v_1 - v_\Gamma \\ v_2 - v_\Gamma \end{bmatrix} 
$$
and thus in component form 
$$
\begin{bmatrix} n_1 \cdot A_1 \nabla u_1 \\ n_2 \cdot A_2 \nabla u_2 \end{bmatrix} + B \begin{bmatrix} u_1 - u_\Gamma \\ u_2 - u_\Gamma \end{bmatrix} = \begin{bmatrix} 0 \\ 0 \end{bmatrix}
$$

# 2. Standard formulation
The standard finite element method takes the form: find \( u_h = (u_{h,1}, u_{h,2}, u_{h,\Gamma}) \in V_h = V_{h,1} \oplus V_{h,2} \oplus V_{h,\Gamma} \) such that
$$ \mathcal{A}_h^S(u_h, v) = L(v) \quad \forall v \in V_h $$

Here the form $\mathcal{A}_h^S$ is defined by
$$ \mathcal{A}_h^S = \mathcal{A} + s_h $$ 

- $A(\cdot, \cdot)$ and $L(\cdot)$ are defined as follows:
$$
\begin{equation}
\begin{aligned}
\mathcal{A}(u, v) &= \sum_{i=1}^2 (A_i \nabla u_i, \nabla v_i)_{\Omega_i}  \\
&\quad + (A_\Gamma \nabla_\Gamma u_\Gamma, \nabla_\Gamma v_\Gamma)_\Gamma + (B(u - u_\Gamma), v - v_\Gamma)_\Gamma \\
L(v) &= \sum_{i=1}^2 (f_i, v_i)_{\Omega_i} + (f_\Gamma, v_\Gamma)_\Gamma 
\end{aligned}
\end{equation}
$$

- $s_h$ is a stabilization term of the form
$$ s_h = s_{h,1} + s_{h,2} + s_{h,\Gamma}  $$
with
$$ s_{h,i}(v, w) = \sum_{F \in \mathcal{F}_{h,i}} h_F \|\zeta(A_i)\|_{\infty,F} \langle [n \cdot \nabla v], [n \cdot \nabla w] \rangle_F \quad i = 1, 2 $$
where $\zeta(X)$ denotes the maximum eigenvalue of the matrix $X$,
$$ s_{h,\Gamma}(v, w) = \sum_{F \in \mathcal{F}_{h,\Gamma}} h_F \|\zeta(A_\Gamma)\|_{\infty,F \cap \Gamma} \langle [n \cdot \nabla v], [n \cdot \nabla w] \rangle_{\mathcal{F}_{h,\Gamma}} $$
$$  \quad + \sum_{T \in \mathcal{T}_{h,\Gamma}} h_K^2 \|\zeta(A_\Gamma)\|_{\infty,K \cap \Gamma} \langle n_\Gamma \cdot \nabla v, n_\Gamma \cdot \nabla w \rangle_{T \cap \Gamma} $$
where for a face sharing two elements $T_1$ and $T_2$ the jump $[n \cdot \nabla v]$ is defined by
$$ [n \cdot \nabla v] = n_1 \cdot \nabla v_1 + n_2 \cdot \nabla v_2 $$
where $n_i$ is the exterior normal to $\partial T_i$ and $v_i = v|_{T_i}$.

In [1]:
from math import pi 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [2]:
# Important parameters and functions
## choose A1 = A2 = A_gamma = I 
## choose alpha = xi = 1, then B is also an identity matrix
f = [2*x*(1-x)+2*y*(1-y), 2*x*(1-x)+2*y*(1-y),1/2]
gD = 0
exact_u_p_neg = x*(1-x)*y*(1-y)
exact_u_p_pos = x*(1-x)*y*(1-y)
exact_u_f = y*(1-y)/4

In [82]:
# Construct background mesh
geo = OCCGeometry(unit_square_shape.Scale((0,0,0),1), dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.4,quad_dominated=False))

In [83]:
# Define the levelset function for the interface
levelset = x-1/2
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [84]:
# Construct the unfitted fem space 
Vhbase = H1(mesh,order=1,dirichlet='.*',dgjumps=True)
ci = CutInfo(mesh,lsetp1)
Vh = FESpace([Compress(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS,IF]])
# Vh = FESpace([Restrict(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS,IF]])

In [87]:
# Visualize the dofs
freedofs = Vh.FreeDofs()
gfu = GridFunction(Vh)
gfu.components[0].Set(1)
gfu.components[1].Set(-1)
gfu.components[2].Set(10)
DrawDC(lsetp1, gfu.components[0], gfu.components[1], mesh, "u")
Draw(gfu.components[2],mesh)
print(Vhbase.ndof, Vh.components[0].ndof, Vh.components[1].ndof,Vh.components[2].ndof)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

TypeError: Draw(): incompatible function arguments. The following argument types are supported:
    1. (cf: ngsolve.fem.CoefficientFunction, mesh: ngsolve.comp.Mesh, name: str, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, draw_vol: bool = True, draw_surf: bool = True, reset: bool = False, title: str = '', number_format: str = '%.3e', unit: str = '', **kwargs) -> None
    2. (gf: ngsolve.comp.GridFunction, sd: int = 2, autoscale: bool = True, min: float = 0.0, max: float = 1.0, **kwargs) -> None
    3. (mesh: ngsolve.comp.Mesh, **kwargs) -> None
    4. (arg0: object) -> None

Invoked with: <ngsolve.comp.ComponentGridFunction object at 0x796b3c11b6b0>, <ngsolve.comp.Mesh object at 0x796b3c57e5d0>

In [86]:
print(len(freedofs))
dofs = GetDofsOfElements(Vhbase, ci.GetElementsOfType(IF)) # 与界面相交的单元包含的自由度
print(dofs.NumSet())

26
6


In [88]:
# Define averages and jumps

u,v = Vh.TnT()
h = specialcf.mesh_size
n = specialcf.normal(2)
jump_grad_u0 = (grad(u[0]) - grad(u[0].Other()))*n
jump_grad_v0 = (grad(v[0]) - grad(v[0].Other()))*n
jump_grad_u1 = (grad(u[1]) - grad(u[1].Other()))*n
jump_grad_v1 = (grad(v[1]) - grad(v[1].Other()))*n
jump_grad_u2 = (grad(u[2]) - grad(u[2].Other()))*n
jump_grad_v2 = (grad(v[2]) - grad(v[2].Other()))*n
jump_u = (grad(u[0]) - grad(u[1]))*n
jump_v = (grad(v[0]) - grad(v[1]))*n

In [89]:
partialOmega1 = x*(x-1/2)*y*(1-y)
pO1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(partialOmega1,pO1)
# DrawDC(pO1,-1,1,mesh,'pO1')
c1 = CutInfo(mesh, pO1)
boundaryEle1 = c1.GetElementsOfType(IF)
# Draw(BitArrayCF(boundaryEle1), mesh, "boundaryelements")

partialOmega2 = (1-x)*(1/2-x)*y*(1-y)
pO2 = GridFunction(H1(mesh,order=1))
InterpolateToP1(partialOmega2,pO2)
c2 = CutInfo(mesh, pO2)
boundaryEle2 = c2.GetElementsOfType(IF)
# Draw(BitArrayCF(boundaryEle2), mesh, "boundaryelements")

In [90]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
haspos = ci.GetElementsOfType(HASPOS)
hasif = ci.GetElementsOfType(IF)

# # facets used for stabilization:
fh1_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=boundaryEle1,bnd_val_a=False,bnd_val_b=False,use_and=True)
fh2_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=boundaryEle2,bnd_val_a=False,bnd_val_b=False,use_and=True)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasif, b=hasif, bnd_val_a=False,bnd_val_b=False,use_and=True) # all interior faces in T_{h,Gamma}

In [97]:
# ba_surround_facets = GetElementsWithNeighborFacets(mesh,fh2_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets")   
print(fh1_facets)
print(fh2_facets)
print(ba_facets)

0: 000000000111000000001110111
0: 000000000010110111011000001
0: 000000000010000000001000001


In [95]:
for v in mesh.vertices:
    print(v,v.point)
for el in mesh.Elements(VOL):
    # print("vertices:",el.vertices)
    # print(type(el.vertices[0]))
    print("edges:",el.edges)
    # print(type(el.edges[0]))

V0 (0.0, 0.0)
V1 (1.0, 0.0)
V2 (1.0, 1.0)
V3 (0.0, 1.0000000000000002)
V4 (0.3333333333333333, 0.0)
V5 (0.6666666666666666, 0.0)
V6 (1.0, 0.3333333333333333)
V7 (1.0, 0.6666666666666666)
V8 (0.6666666666666667, 1.0)
V9 (0.33333333333333337, 1.0000000000000002)
V10 (-6.123233995736765e-17, 0.666666666666667)
V11 (-1.224646799147353e-16, 0.3333333333333336)
V12 (0.6609976134160176, 0.43923354093144445)
V13 (0.3390023860074996, 0.5607664577718635)
edges: (E11, E26, E10)
edges: (E19, E26, E20)
edges: (E2, E12, E3)
edges: (E15, E17, E14)
edges: (E9, E25, E11)
edges: (E20, E22, E18)
edges: (E6, E21, E7)
edges: (E24, E25, E23)
edges: (E13, E15, E12)
edges: (E4, E16, E5)
edges: (E22, E24, E21)
edges: (E1, E9, E0)
edges: (E10, E13, E8)
edges: (E17, E19, E16)


In [43]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))
df0 = dFacetPatch(definedonelements=fh1_facets)
df1 = dFacetPatch(definedonelements=fh2_facets)
df2 = dFacetPatch(definedonelements=ba_facets)
## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += grad(u[0]) * grad(v[0]) * dx_neg +  grad(u[1]) * grad(v[1]) * dx_pos + grad(u[2]) * grad(v[2]) * ds
ah += ((u[0]-u[2]) * (v[0]-v[2]) + (u[1]-u[2]) * (v[1]-v[2]) )* ds
# stabilization terms
ah += h * jump_grad_u0 * jump_grad_v0 * df0
ah += h * jump_grad_u1 * jump_grad_v1 * df1
ah += (h * jump_grad_u2 * jump_grad_v2 * df2 + h * jump_u * jump_v * ds)
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos + f[2] * v[2] * ds)
F.Assemble()


In [13]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
ds = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))
df0 = dFacetPatch(definedonelements=fh1_facets)
df1 = dFacetPatch(definedonelements=fh2_facets)
df2 = dFacetPatch(definedonelements=ba_facets)
## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += grad(u[0]) * grad(v[0]) * dx_neg +  grad(u[1]) * grad(v[1]) * dx_pos + grad(u[2]) * grad(v[2]) * ds
ah += ((u[0]-u[2]) * (v[0]-v[2]) + (u[1]-u[2]) * (v[1]-v[2]) )* ds
# stabilization terms
ah += h * jump_grad_u0 * jump_grad_v0 * dx(skeleton=True,definedonelements=fh1_facets)
ah += h * jump_grad_u1 * jump_grad_v1 * dx(skeleton=True,definedonelements=fh2_facets)
ah += (h * jump_grad_u2 * jump_grad_v2 * dx(skeleton=True,definedonelements=ba_facets) + h * jump_u * jump_v * dx(skeleton=True,definedonelements=ci.GetElementsOfType(IF)))
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos + f[2] * v[2] * ds)
F.Assemble()

In [81]:
from ngsolve import *

Vf = FacetFESpace(mesh, order=0, dirichlet=[])   # 常数向量场
n = specialcf.normal(2)
gf = GridFunction(Vf)

gf.Set(n)
Draw(gf, mesh, "n_facet")


NgException: Error in SetValues: gridfunction-dim = 1, but coefficient-dim = 2

In [74]:
# # rl = jump_v*dx(skeleton=True,definedonelements=ci.GetElementsOfType(IF))
# rl = jump_grad_v0*df0
# ff = LinearForm(rl).Assemble()
# print(ff.vec.Norm())
ju = u[0] - u[0].Other()
jv = v[0] - v[0].Other()
a = BilinearForm(Vh)
a += (grad(u[0]) - grad(u[0].Other())) * n * (grad(v[0]) - grad(v[0].Other())) * n * df0
a.Assemble()
# print(a.mat)

for f in ba_facets:
    facet = mesh[f]
    print(f"Facet {f}, normal = {facet.Normal()}")


used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


AttributeError: 'ngsolve.comp.Mesh' object has no attribute 'GetFacet'

In [44]:
# Solve the system

gfu = GridFunction(Vh)
freedofs = Vh.FreeDofs()
## deal with Dirichlet boundary
gfu.components[0].Set(gD,BND)
gfu.components[1].Set(gD,BND)
F.vec.data -= ah.mat * gfu.vec
gfu.vec.data += ah.mat.Inverse(freedofs) * F.vec


In [45]:
# Calculate L2 error
# u_exact = [exact_u_neg, exact_u_pos]
err = sqrt(Integrate((gfu.components[0] - exact_u_p_neg)**2*dx_neg, mesh) + Integrate((gfu.components[1] - exact_u_p_pos)**2*dx_pos, mesh)) 
print(err)
err_f = sqrt(Integrate((gfu.components[2] - exact_u_f)**2*ds, mesh))
print(err_f)

0.0006823580880380291
0.00044887009995896486


In [46]:
DrawDC(levelset, gfu.components[0], gfu.components[1], mesh, "u")
DrawDC(levelset, exact_u_p_neg, exact_u_p_pos, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [52]:
exact_u = x*(1-x)*y*(1-y)
Draw(exact_u,mesh)

gfu_orig = GridFunction(Vhbase)
gfu_orig.vec[:] = 0  # 默认设置为 0
used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASNEG))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[0].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASPOS))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[1].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(IF))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] -= gfu.components[2].vec[i]

Draw(gfu_orig,mesh)

error = sqrt(Integrate((gfu_orig - exact_u)**2*dx, mesh))
print(error)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

0.0005728990415575403


In [47]:
Draw(gfu.components[0], mesh, "u_neg")
Draw(gfu.components[1], mesh, "u_pos")
Draw(exact_u_p_neg, mesh, "exact_neg")
Draw(exact_u_p_pos, mesh, "exact_pos")


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [20]:
Draw(gfu.components[2], mesh, "u_f")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene